In [132]:
import numpy as np
import pandas as pd

In [133]:
from bs4 import BeautifulSoup

In [134]:
import requests

page = requests.get(r'https://www.espncricinfo.com/records/highest-career-batting-average-282910')

soup1 = BeautifulSoup(page.content, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [135]:
tables = soup2.find_all('table')
len(tables)

1

In [136]:
table = tables[0]

In [137]:
import io

table_io = io.StringIO(str(table))
df = pd.read_html(table_io)

In [138]:
df = pd.DataFrame(df[0])

In [139]:
df.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,DG Bradman (AUS),1928-1948,52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6
1,HC Brook (ENG),2022-2023,12,20,1,1181,186,62.15,1287,91.76,4,7,1,141,23
2,AC Voges (AUS),2015-2016,20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5
3,RG Pollock (SA),1963-1970,23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11
4,GA Headley (WI),1930-1954,22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1


In [140]:
df = df.rename(columns={'NO':'not_out', 'HS':'highest_inns_score',
                         'Mat':'matches_played', 'Ave':'batting_average',
                         'Runs':'runs_scored', 'SR':'batting_strike_rate',
                         '100':'hundreds_scored', '50':'fifties_scored',
                         '0':'ducks_scored', '4s':'boundary_fours',
                         '6s':'boundary_sixes', 'BF':'balls_faced'
                         })

df.head()

,Player,Span,matches_played,Inns,not_out,runs_scored,highest_inns_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties_scored,ducks_scored,boundary_fours,boundary_sixes
0,DG Bradman (AUS),1928-1948,52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6
1,HC Brook (ENG),2022-2023,12,20,1,1181,186,62.15,1287,91.76,4,7,1,141,23
2,AC Voges (AUS),2015-2016,20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5
3,RG Pollock (SA),1963-1970,23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11
4,GA Headley (WI),1930-1954,22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1


In [141]:
df.isna().sum()

Player                 0
Span                   0
matches_played         0
Inns                   0
not_out                0
runs_scored            0
highest_inns_score     0
batting_average        0
balls_faced            0
batting_strike_rate    0
hundreds_scored        0
fifties_scored         0
ducks_scored           0
boundary_fours         0
boundary_sixes         0
dtype: int64

In [142]:
df['Player'].duplicated().any()

False

In [143]:
df['Span'].str.len().value_counts()

Span
9    63
Name: count, dtype: int64

In [144]:
span_series = df['Span'].str.split('-')
span_series[:5]

0    [1928, 1948]
1    [2022, 2023]
2    [2015, 2016]
3    [1963, 1970]
4    [1930, 1954]
Name: Span, dtype: object

In [145]:
def get_start(x):
    return x[0]

def get_end(x):
    return x[1]

df['start_career'] = span_series.apply(get_start)
df['end_career'] = span_series.apply(get_end)
df.head()

,Player,Span,matches_played,Inns,not_out,runs_scored,highest_inns_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties_scored,ducks_scored,boundary_fours,boundary_sixes,start_career,end_career
0,DG Bradman (AUS),1928-1948,52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6,1928,1948
1,HC Brook (ENG),2022-2023,12,20,1,1181,186,62.15,1287,91.76,4,7,1,141,23,2022,2023
2,AC Voges (AUS),2015-2016,20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5,2015,2016
3,RG Pollock (SA),1963-1970,23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11,1963,1970
4,GA Headley (WI),1930-1954,22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1,1930,1954


In [146]:
df.drop(columns='Span', inplace=True)

In [147]:
(df['Player'].str.find('(') == -1).value_counts()

Player
False    63
Name: count, dtype: int64

In [148]:
import re

pattern = r'\((.+)\)'

df['nation'] = df['Player'].str.findall(pattern, flags=re.IGNORECASE).str.get(0)
df.head()

,Player,matches_played,Inns,not_out,runs_scored,highest_inns_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties_scored,ducks_scored,boundary_fours,boundary_sixes,start_career,end_career,nation
0,DG Bradman (AUS),52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6,1928,1948,AUS
1,HC Brook (ENG),12,20,1,1181,186,62.15,1287,91.76,4,7,1,141,23,2022,2023,ENG
2,AC Voges (AUS),20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5,2015,2016,AUS
3,RG Pollock (SA),23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11,1963,1970,SA
4,GA Headley (WI),22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1,1930,1954,WI


In [149]:
pattern2 = r'(\w*\s\w+)\s'

df['Player'] = df['Player'].str.findall(pattern2, flags=re.IGNORECASE).str.get(0)
df.tail(3)

,Player,matches_played,Inns,not_out,runs_scored,highest_inns_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties_scored,ducks_scored,boundary_fours,boundary_sixes,start_career,end_career,nation
60,WH Ponsford,29,48,4,2122,266,48.22,3118+,44.77,7,6,1,119+,0,1924,1934,AUS
61,SJ McCabe,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+,1930,1938,AUS
62,DR Jardine,22,33,6,1296,127,48.00,2110+,25.59,1,10,2,53+,0,1928,1934,ENG


In [150]:
df.dtypes

Player                  object
matches_played           int64
Inns                     int64
not_out                  int64
runs_scored              int64
highest_inns_score      object
batting_average        float64
balls_faced             object
batting_strike_rate     object
hundreds_scored          int64
fifties_scored           int64
ducks_scored             int64
boundary_fours          object
boundary_sixes          object
start_career            object
end_career              object
nation                  object
dtype: object

In [151]:
def is_num(x):
    return x.isnumeric()

df['highest_inns_score'].apply(is_num).value_counts()

highest_inns_score
True     43
False    20
Name: count, dtype: int64

In [152]:
mask = df['highest_inns_score'].apply(is_num)

df.loc[~mask, 'highest_inns_score']

2     269*
4     270*
9     336*
11    365*
21    247*
23    248*
25    400*
26    280*
32    201*
33    232*
35    203*
36    236*
39    278*
47    182*
49    254*
50    215*
51    329*
52    144*
53    189*
54    144*
Name: highest_inns_score, dtype: object

In [153]:
pattern = '\d+'

df['max_inns_score'] = df['highest_inns_score'].str.findall(pattern).str.get(0)
df['max_inns_score'] = df['max_inns_score'].astype('int')
df['max_inns_score'][:5]

0    334
1    186
2    269
3    274
4    270
Name: max_inns_score, dtype: int32

In [154]:
df.drop(columns='highest_inns_score', inplace=True)

## balls_faced

In [155]:
df['balls_faced'].apply(is_num).value_counts()

balls_faced
False    33
True     30
Name: count, dtype: int64

In [156]:
mask = df['balls_faced'].apply(is_num)

df.loc[~mask, 'balls_faced'][:5]

0    9800+
3    1707+
4     416+
5    6558+
6    1288+
Name: balls_faced, dtype: object

In [157]:
df['balls_faced'].str.len().value_counts()

balls_faced
5    40
4    15
6     5
1     3
Name: count, dtype: int64

In [158]:
mask2 = (df['balls_faced'].str.len() == 1)

df.loc[mask2, 'balls_faced']

8     -
14    -
54    -
Name: balls_faced, dtype: object

In [159]:
pattern = r'\d+'

df = df.rename(columns={'balls_faced':'balls_faced2'})
df['balls_faced'] = df['balls_faced2'].str.findall(pattern).str.get(0)
df['balls_faced'][:9]

0    9800
1    1287
2    2667
3    1707
4     416
5    6558
6    1288
7    4957
8     NaN
Name: balls_faced, dtype: object

In [160]:
df['balls_faced'] = df['balls_faced'].astype('Int32')
df['balls_faced'][:9]

0    9800
1    1287
2    2667
3    1707
4     416
5    6558
6    1288
7    4957
8    <NA>
Name: balls_faced, dtype: Int32

In [161]:
df.drop(columns='balls_faced2',inplace=True)
df.dtypes

Player                  object
matches_played           int64
Inns                     int64
not_out                  int64
runs_scored              int64
batting_average        float64
batting_strike_rate     object
hundreds_scored          int64
fifties_scored           int64
ducks_scored             int64
boundary_fours          object
boundary_sixes          object
start_career            object
end_career              object
nation                  object
max_inns_score           int32
balls_faced              Int32
dtype: object

## batting_strike_rate

In [162]:
df['batting_strike_rate'].apply(is_num).value_counts()

batting_strike_rate
False    63
Name: count, dtype: int64

In [163]:
df['batting_strike_rate'][0:5]

0    58.60
1    91.76
2    55.68
3    54.48
4    56.00
Name: batting_strike_rate, dtype: object

In [164]:
df['batting_strike_rate'].str.len().value_counts()

batting_strike_rate
5    60
1     3
Name: count, dtype: int64

In [165]:
mask = (df['batting_strike_rate'].str.len() == 1)
df.loc[mask, 'batting_strike_rate']

8     -
14    -
54    -
Name: batting_strike_rate, dtype: object

In [166]:
df.loc[mask, 'batting_strike_rate'] = np.nan
df['batting_strike_rate'] = df['batting_strike_rate'].astype('float32')

## boundary_fours

In [167]:
df['boundary_fours'].str.len().value_counts()

boundary_fours
4    41
3    21
5     1
Name: count, dtype: int64

In [168]:
df['boundary_fours'][0:5]

0    626+
1     141
2     186
3    246+
4    104+
Name: boundary_fours, dtype: object

In [169]:
pattern = '\d+'

df['boundary_fours'] = df['boundary_fours'].str.findall(pattern).str.get(0)
df['boundary_fours'] = df['boundary_fours'].astype(int)
df['boundary_fours'][0:5]

0    626
1    141
2    186
3    246
4    104
Name: boundary_fours, dtype: int32

## boundary_sixes

In [171]:
df['boundary_sixes'].str.len().value_counts()

boundary_sixes
2    40
1    22
3     1
Name: count, dtype: int64

In [172]:
df['boundary_sixes'][:5]

0     6
1    23
2     5
3    11
4     1
Name: boundary_sixes, dtype: object

In [174]:
df = df.rename(columns={'boundary_sixes':'boundary_sixes2'})

pattern = '\d+'

df['boundary_sixes'] = df['boundary_sixes2'].str.findall(pattern).str.get(0)
df['boundary_sixes'] = df['boundary_sixes'].astype(int)
df['boundary_sixes'][0:5]

0     6
1    23
2     5
3    11
4     1
Name: boundary_sixes, dtype: int32

In [176]:
df['boundary_sixes'].isnull().any()

False

In [177]:
df.drop(columns='boundary_sixes2', inplace=True)

In [179]:
df['start_career'] = df['start_career'].astype(int)
df['end_career'] = df['end_career'].astype(int)

In [180]:
df.dtypes

Player                  object
matches_played           int64
Inns                     int64
not_out                  int64
runs_scored              int64
batting_average        float64
batting_strike_rate    float32
hundreds_scored          int64
fifties_scored           int64
ducks_scored             int64
boundary_fours           int32
start_career             int32
end_career               int32
nation                  object
max_inns_score           int32
balls_faced              Int32
boundary_sixes           int32
dtype: object

In [181]:
df['career_length'] = df['end_career'] - df['start_career']
df.head(3)

,Player,matches_played,Inns,not_out,runs_scored,batting_average,batting_strike_rate,hundreds_scored,fifties_scored,ducks_scored,boundary_fours,start_career,end_career,nation,max_inns_score,balls_faced,boundary_sixes,career_length
0,DG Bradman,52,80,10,6996,99.94,58.599998,29,13,7,626,1928,1948,AUS,334,9800,6,20
1,HC Brook,12,20,1,1181,62.15,91.760002,4,7,1,141,2022,2023,ENG,186,1287,23,1
2,AC Voges,20,31,7,1485,61.87,55.680000,5,4,2,186,2015,2016,AUS,269,2667,5,1


In [183]:
df.columns.to_list()

['Player',
 'matches_played',
 'Inns',
 'not_out',
 'runs_scored',
 'batting_average',
 'batting_strike_rate',
 'hundreds_scored',
 'fifties_scored',
 'ducks_scored',
 'boundary_fours',
 'start_career',
 'end_career',
 'nation',
 'max_inns_score',
 'balls_faced',
 'boundary_sixes',
 'career_length']

In [187]:
sorted_col = [
    'Player',
    'nation',
    'start_career',
    'end_career',
    'career_length',
    'matches_played',
    'Inns',
    'max_inns_score',
    'balls_faced',
    'boundary_sixes',
    'not_out',
    'runs_scored',
    'batting_average',
    'batting_strike_rate',
    'hundreds_scored',
    'fifties_scored',
    'ducks_scored',
    'boundary_fours',
]

df = df[sorted_col]

In [188]:
df.head()

,Player,nation,start_career,end_career,career_length,matches_played,Inns,max_inns_score,balls_faced,boundary_sixes,not_out,runs_scored,batting_average,batting_strike_rate,hundreds_scored,fifties_scored,ducks_scored,boundary_fours
0,DG Bradman,AUS,1928,1948,20,52,80,334,9800,6,10,6996,99.94,58.599998,29,13,7,626
1,HC Brook,ENG,2022,2023,1,12,20,186,1287,23,1,1181,62.15,91.760002,4,7,1,141
2,AC Voges,AUS,2015,2016,1,20,31,269,2667,5,7,1485,61.87,55.680000,5,4,2,186
3,RG Pollock,SA,1963,1970,7,23,41,274,1707,11,4,2256,60.97,54.480000,7,11,1,246
4,GA Headley,WI,1930,1954,24,22,40,270,416,1,4,2190,60.83,56.000000,10,5,2,104
